# DBF file to MySQL for Int File

## 1. Load DBF to Pandas DataFrame

In [85]:
from datetime import datetime
import pandas as pd
print(f'pandas version: {pd.__version__}')

pd.set_option('max_columns', 500)
# pd.set_option('max_rows', 10)

pandas version: 1.0.1


In [86]:
"""
Load content of a DBF file into a Pandas data frame.

The iter() is required because Pandas doesn't detect that the DBF
object is iterable.
"""

from dbfread import DBF

In [87]:
setPath = input('Please enter path for int file (default as G:/relia/custbase/): ') or 'G:/relia/custbase/'
yearSelected = input('Please insert year (default as this year) : ') or str(datetime.now().year)
print(f'Path : {setPath}')
print(f'Year : {yearSelected}')
fullPath = setPath + 'int_'+ yearSelected +'.dbf'
print(f'\nFull path : {fullPath}')

Please enter path for int file (default as G:/relia/custbase/): 
Please insert year (default as this year) : 
Path : G:/relia/custbase/
Year : 2020

Full path : G:/relia/custbase/int_2020.dbf


In [88]:
dbf = DBF(fullPath)
df = pd.DataFrame(iter(dbf))

In [89]:
total_records = df.shape[0]
df.tail()

,ID,DATE,NEW_MONTH,FROM,TO,ABB,FEEDER,LINE,CB,TIMEOCB,GROUP,EVENT,OUTGDIST,CUSTDIST,AREA,NIKOM,TYPE,NEW_CODE,RELAY,COMPONENT,ROAD,LATERAL,TIME_EQ,CUST_NUM,CUST_MIN,MAJOR,CONTROL,WEATHER,BK,BK_MIN,SP,SP_MIN,NB,NB_MIN
14641,774.0,2020-10-18,10,19:24,19:24,PA,PA-424,,,0,F,O,1,1,C,N,CO,191,"OCI,EFI,RR",C07,,HT.33-01632,0.0,2468.0,0.0,,,N,2468.0,0.0,0.0,0.0,0.0,0.0
14642,775.0,2020-10-18,10,20:22,20:22,KU,KU-427,,,0,F,I,7,7,C,N,TF,011,"OCI-B,EFI,RR",,,,0.0,4786.0,0.0,,,N,3336.0,0.0,1450.0,0.0,0.0,0.0
14643,776.0,2020-10-18,10,23:10,23:10,CK,CK-424,,,0,F,I,14,14,C,N,TF,053,"OCI-T-Y,EFI-T,RR",C07,,SOI LARDPHRAO 1/18,0.0,4940.0,0.0,,,N,4940.0,0.0,0.0,0.0,0.0,0.0
14644,777.0,2020-10-18,10,23:53,23:53,SS,SS-425,,,0,F,I,1,1,C,N,TF,011,"OCI-T-Y,EFI-T,RR",,,,0.0,2939.0,0.0,,,N,2939.0,0.0,0.0,0.0,0.0,0.0
14645,777.0,2020-10-18,10,23:53,23:53,SS,SS-425,,,0,F,I,1,8,C,N,TF,011,"OCI-T-Y,EFI-T,RR",,,,0.0,4233.0,0.0,,,N,4233.0,0.0,0.0,0.0,0.0,0.0


## - Import module for connecting to MySQL

In [90]:
from sqlalchemy import create_engine # conda install sqlalchemy
import pymysql # conda install pymysql

In [91]:
print(f'pymysql version: {pymysql.__version__}')

pymysql version: 0.10.1


### - create a connection to MySQL

In [92]:
# uid = 'reliability'
# pwd = 'pcd_db'
# host = '10.99.1.36'
# uid = 'root'
# pwd = 'admin'
# host = 'localhost'
uid = 'root'
pwd = ''
host = 'localhost'
port = 3306
db = 'statistics_database'
con_string = f'mysql+pymysql://{uid}:{pwd}@{host}:{port}/{db}'
print(f'connection string = {con_string}')
con = create_engine(con_string)

connection string = mysql+pymysql://root:@localhost:3306/statistics_database


In [93]:
# con.table_names()

## 2. Change data type in each fields

In [94]:
# import Javascript for working with browser
# from IPython.display import Javascript
# Javascript("alert();")

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14646 entries, 0 to 14645
Data columns (total 34 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         14645 non-null  float64
 1   DATE       14646 non-null  object 
 2   NEW_MONTH  14646 non-null  int64  
 3   FROM       14646 non-null  object 
 4   TO         14646 non-null  object 
 5   ABB        14646 non-null  object 
 6   FEEDER     14646 non-null  object 
 7   LINE       14646 non-null  object 
 8   CB         14646 non-null  object 
 9   TIMEOCB    14646 non-null  int64  
 10  GROUP      14646 non-null  object 
 11  EVENT      14646 non-null  object 
 12  OUTGDIST   14646 non-null  int64  
 13  CUSTDIST   14646 non-null  int64  
 14  AREA       14646 non-null  object 
 15  NIKOM      14646 non-null  object 
 16  TYPE       14646 non-null  object 
 17  NEW_CODE   14646 non-null  object 
 18  RELAY      14646 non-null  object 
 19  COMPONENT  14646 non-null  object 
 20  ROAD  

### 2.1 DATE field to datetime

In [96]:
regex = r'[12]\d{3}-0[1-9]|1[0-2]-0[1-9]|[12]\d|3[01]'
df[~(df['DATE'].astype(str).str.contains(regex))]['DATE'].unique()

array([], dtype=object)

In [97]:
df['DATE'] = pd.to_datetime(df['DATE'], errors='coerce')
# df.info()
print(f'Total records : {total_records}')
print('NaT records : %d' %df[df['DATE'].isna()]['ID'].count())
print('Date records : %d' %df[~df['DATE'].isna()]['ID'].count())
print('SUM records : %d' %(df[df['DATE'].isna()]['ID'].count() + df[~df['DATE'].isna()]['ID'].count()))

Total records : 14646
NaT records : 0
Date records : 14645
SUM records : 14645


In [98]:
# df.dtypes.value_counts()

## 3. Check miss match data in each fields

In [99]:
# df.info()

### 3.1 Weather

In [100]:
# df.loc[0:5, 'WEATHER'] = 'S'
indexList = df[~df['WEATHER'].isin(['N', 'R'])].index
if len(indexList):
    print(df.loc[indexList, 'WEATHER'])
    df.loc[indexList, 'WEATHER'] = 'N'
    print(df.loc[indexList, 'WEATHER'])
else:
    print('PASS^^ : Weahter data have already complete.')

PASS^^ : Weahter data have already complete.


### 3.2 ABB

In [101]:
# Check empty ABB (ABB can empty if GROUP is 'E')
# pd.crosstab(df['ABB'], columns='COUNT')
if len(df[(df['ABB'] == '') & (df['GROUP'] != 'E')]) > 0:
    print('Please check empty ABB records !!!')
else:
    print('Pass^^ : No empty ABB records for non "E" GROUP')

Pass^^ : No empty ABB records for non "E" GROUP


In [102]:
# %%time
# Check ABB is not in Feeder fields
# df[df['GROUP'].isin(['F'])][['ABB', 'FEEDER']].apply(lambda row: print('Miss match "ABB" : %s and "FEEDER" : %s' %(row['ABB'], row['FEEDER']) if row['ABB'] not in row['FEEDER'] else 'Match'), axis=1);
missmatchAbbFeeder = pd.DataFrame()
feederRows = df[df['GROUP'].isin(['F'])][['ABB', 'FEEDER']]
for idx, row in feederRows.iterrows():
    if row['ABB'] not in row['FEEDER']:
        missmatchAbbFeeder = missmatchAbbFeeder.append(row)

if len(missmatchAbbFeeder) > 0:
    print('Please check group "F" records that "ABB" miss match with "FEEDER" !!!')
    print(missmatchAbbFeeder)
    missmatchAbbFeederIdx = missmatchAbbFeeder.index
    for idx, row in missmatchAbbFeeder.iterrows():
        df.loc[idx, 'ABB'] = row['FEEDER'].split('-',1)[0]
    print('They have already corrected^^')
    print(df.loc[missmatchAbbFeederIdx, ['ABB', 'FEEDER']])
else:
    print('Pass^^ : All group "F" records, "ABB" match with "FEEDER"')

Please check group "F" records that "ABB" miss match with "FEEDER" !!!
      ABB   FEEDER
4360  PEW  PWE-421
They have already corrected^^
      ABB   FEEDER
4360  PWE  PWE-421


In [103]:
# Check ABB is not in CB fields
missmatchAbbStaionH = pd.DataFrame() # StationH = Station and 'H' event
missmatchAbbStationF = pd.DataFrame() # StationF = Station and 'I, O' event
stationRows = df[df['GROUP'].isin(['S'])][['ABB', 'FEEDER', 'CB', 'EVENT']]
for idx, row in stationRows.iterrows():
    if row['EVENT'] == 'H':
        if row['ABB'] not in row['CB']:
            missmatchAbbStaionH = missmatchAbbStaionH.append(row)
    else: # row['EVENT'] != 'H'
        if row['ABB'] not in row['FEEDER']:
            missmatchAbbStationF = missmatchAbbStationF.append(row)

if len(missmatchAbbStaionH) > 0:
    print('Please check group "S" records that "ABB" miss match with "CB" !!!')
    print(missmatchAbbStaionH)
else:
    print('Pass^^ : All group "S" records, "ABB" match with "CB"')
    
if len(missmatchAbbStationF) > 0:
    print('Please check group "S" records that "ABB" miss match with "FEEDER" !!!')
    print(missmatchAbbStationF)
    missmatchAbbStationFIdx = missmatchAbbStationF.index
    for idx, row in missmatchAbbStationF.iterrows():
        df.loc[idx, 'ABB'] = row['FEEDER'].split('-',1)[0]
    print('They have already corrected^^')
    print(df.loc[missmatchAbbStationFIdx, ['ABB', 'CB', 'EVENT', 'FEEDER']])
else:
    print('Pass^^ : All group "S" records, "ABB" match with "FEEDER"')

Pass^^ : All group "S" records, "ABB" match with "CB"
Pass^^ : All group "S" records, "ABB" match with "FEEDER"


In [104]:
# df.loc[12507:12509, 'ABB'] = 'WPC'

In [105]:
# Check ABB is not in LINE fields
missmatchAbbLineH = pd.DataFrame() # LineH = Line and 'H' event
missmatchAbbLineF = pd.DataFrame() # LineH = Line and 'I, O' event
lineRows = df[df['GROUP'].isin(['L'])][['ABB', 'FEEDER', 'LINE', 'EVENT']]
for idx, row in lineRows.iterrows():
    if row['EVENT'] == 'H':
        if row['ABB'] not in row['LINE']:
            missmatchAbbLineH = missmatchAbbLineH.append(row)
    else: # row['EVENT'] != 'H'
        if row['ABB'] not in row['FEEDER']:
            missmatchAbbLineF = missmatchAbbLineF.append(row)

if len(missmatchAbbLineH) > 0:
    print('Please check group "L" records that "ABB" miss match with "LINE" !!!')
    print(missmatchAbbLineH)
else:
    print('Pass^^ : All group "L" records, "ABB" match with "LINE"')
    
if len(missmatchAbbLineF) > 0:
    print('Please check group "L" records that "ABB" miss match with "FEEDER" !!!')
    print(missmatchAbbLineF)
    missmatchAbbLineFIdx = missmatchAbbLineF.index
    for idx, row in missmatchAbbLineF.iterrows():
        df.loc[idx, 'ABB'] = row['FEEDER'].split('-',1)[0]
    print('They have already corrected^^')
    print(df.loc[missmatchAbbLineFIdx, ['ABB', 'EVENT', 'FEEDER', 'LINE']])
else:
    print('Pass^^ : All group "L" records, "ABB" match with "FEEDER"')

Pass^^ : All group "L" records, "ABB" match with "LINE"
Pass^^ : All group "L" records, "ABB" match with "FEEDER"


### 3.3 FROM

In [106]:
regex = r'^[0-9]|[0-1][0-9]|[2][0-3]:[0-5][0-9]$'
missmatchFrom = df[~df['FROM'].str.contains(regex)]['FROM']
if len(missmatchFrom) > 0:
    print('Please check "FROM" miss match records !!!')
else:
    print('Pass^^ : All "FROM" records have already correct')

Pass^^ : All "FROM" records have already correct


### 3.4 TO

In [107]:
regex = r'^[0-9]|[0-1][0-9]|[2][0-3]:[0-5][0-9]$'
missmatchTo = df[~df['TO'].str.contains(regex)]['TO']
if len(missmatchTo) > 0:
    print('Please check "TO" miss match records !!!')
    missmatchToIdx = missmatchTo.index
    print(missmatchTo)
    df.loc[missmatchToIdx, 'TO'] = None
    print('They have already corrected^^')
    print(df.loc[missmatchToIdx, 'TO'])
else:
    print('Pass^^ : All "TO" records have already correct')

Pass^^ : All "TO" records have already correct


### 3.5 NEW_CODE

In [108]:
# con.table_names()
# df[0:1] -> new_code = '053'
# df.loc[0, 'NEW_CODE'] = '053'

In [109]:
# r = con.execute('select * from nw_cause limit 5;').fetchall()
# r
sql = 'select sub_code from nw_cause'
nw_cause_sub_code = pd.read_sql(sql, con).iloc[:, 0].values
missmatchNew_code = df[~df['NEW_CODE'].isin(nw_cause_sub_code)]\
                        [['DATE', 'FEEDER', 'FROM', 'LINE', 'CB', 'GROUP', 'OUTGDIST', 'CUSTDIST', 'NEW_CODE', 'COMPONENT']]
if len(missmatchNew_code) > 0:
    print('Please check "NEW_CODE" miss match records !!!')
    print(missmatchNew_code)
else:
    print('Pass^^ : All "NEW_CODE" records have already correct')

Pass^^ : All "NEW_CODE" records have already correct


### 3.6 OUTGDIST

In [110]:
# pd.crosstab(df['OUTGDIST'], columns='COUNT')
# df[~df['OUTGDIST'].isin(range(1,19))]['EVENT'].unique()
# df[df['EVENT']=='S']
# 15
# df.loc[0, 'OUTGDIST'] = 15

In [111]:
missmatchOutgdist = df[~df['OUTGDIST'].isin(range(1,19)) & df['EVENT'].isin(['I', 'O'])]\
                        [['DATE', 'FEEDER', 'FROM', 'LINE', 'CB', 'GROUP', 'OUTGDIST', 'CUSTDIST', 'NEW_CODE', 'COMPONENT']]
len(missmatchOutgdist)
if len(missmatchOutgdist) > 0:
    print('Please check "OUTGDIST" miss match records !!!')
    print(missmatchOutgdist)
else:
    print('Pass^^ : All "OUTGDIST" records have already correct')

Pass^^ : All "OUTGDIST" records have already correct


### 3.7 CUSTDIST

In [112]:
# pd.crosstab(df['CUSTDIST'], columns='COUNT')
# df[~df['CUSTDIST'].isin(range(1,19))]['EVENT'].unique()
# df[df['EVENT']=='S']
# 15
# df.loc[0, 'CUSTDIST'] = 15

In [113]:
missmatchCustdist = df[~df['CUSTDIST'].isin(range(1,19)) & df['EVENT'].isin(['I', 'O'])]\
                        [['DATE', 'FEEDER', 'FROM', 'LINE', 'CB', 'GROUP', 'OUTGDIST', 'CUSTDIST', 'NEW_CODE', 'COMPONENT']]
len(missmatchCustdist)
if len(missmatchCustdist) > 0:
    print('Please check "CUSTDIST" miss match records !!!')
    print(missmatchCustdist)
else:
    print('Pass^^ : All "CUSTDIST" records have already correct')

Pass^^ : All "CUSTDIST" records have already correct


### 3.8 COMPONENT

In [114]:
# df.loc[0, 'COMPONENT'] = 'C22' # 'C22'

In [115]:
sql = 'select code from component'
component_code = pd.read_sql(sql, con).iloc[:, 0]
missmatchComponent = df[~((df['COMPONENT'].isin(component_code)) | (df['COMPONENT'] == ''))]\
                        [['DATE', 'FEEDER', 'FROM', 'LINE', 'CB', 'GROUP', 'OUTGDIST', 'CUSTDIST', 'NEW_CODE', 'COMPONENT']]
if len(missmatchComponent) > 0:
    print('Please check "COMPONENT" miss match records !!!')
    print(missmatchComponent)
else:
    print('Pass^^ : All "COMPONENT" records have already correct')

Pass^^ : All "COMPONENT" records have already correct


### 3.9 CONTROL

In [116]:
missmatchControl = df[(df['CONTROL'] == '') | (df['CONTROL'].isna())]\
                        [['DATE', 'FEEDER', 'FROM', 'LINE', 'CB', 'GROUP', 'NEW_CODE', 'COMPONENT', 'CONTROL']]
if len(missmatchControl) > 0:
    print('Please check "CONTROL" miss match records !!!')
    print(missmatchControl)
else:
    print('Pass^^ : All "CONTROL" records have already correct')

Please check "CONTROL" miss match records !!!
            DATE   FEEDER   FROM LINE CB GROUP NEW_CODE COMPONENT CONTROL
13799 2020-10-01  KUS-426  03:29             F      053       C22        
13800 2020-10-01  RTR-411  05:24             F      041       C07        
13801 2020-10-01  RTR-411  05:24             F      041       C07        
13802 2020-10-01  RTB-433  05:53             F      011                  
13803 2020-10-01  RTB-433  05:53             F      011                  
...          ...      ...    ...  ... ..   ...      ...       ...     ...
14641 2020-10-18   PA-424  19:24             F      191       C07        
14642 2020-10-18   KU-427  20:22             F      011                  
14643 2020-10-18   CK-424  23:10             F      053       C07        
14644 2020-10-18   SS-425  23:53             F      011                  
14645 2020-10-18   SS-425  23:53             F      011                  

[847 rows x 9 columns]


## 4. Add CROSSDIST

In [136]:
# df['CROSSDIST'] = 0 if df[df['OUTGDIST'] != df['CUSTDIST']] else 1
df['CROSSDIST'] = df.apply(lambda x: 0 if x['OUTGDIST'] != x['CUSTDIST'] else 1 , axis=1)
df.CROSSDIST.value_counts()

1    12523
0     2123
Name: CROSSDIST, dtype: int64

## 5. Export and check to MySQL

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14646 entries, 0 to 14645
Data columns (total 35 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   ID         14645 non-null  float64       
 1   DATE       14646 non-null  datetime64[ns]
 2   NEW_MONTH  14646 non-null  int64         
 3   FROM       14646 non-null  object        
 4   TO         14646 non-null  object        
 5   ABB        14646 non-null  object        
 6   FEEDER     14646 non-null  object        
 7   LINE       14646 non-null  object        
 8   CB         14646 non-null  object        
 9   TIMEOCB    14646 non-null  int64         
 10  GROUP      14646 non-null  object        
 11  EVENT      14646 non-null  object        
 12  OUTGDIST   14646 non-null  int64         
 13  CUSTDIST   14646 non-null  int64         
 14  AREA       14646 non-null  object        
 15  NIKOM      14646 non-null  object        
 16  TYPE       14646 non-null  object       

In [138]:
# Replace field names
df.columns = pd.read_sql('select * from indices_db_15days limit 1', con).columns[1:]
df.columns

Index(['id', 'date', 'new_month', 'time_from', 'time_to', 'abb', 'feeder',
       'line', 'cb', 'timeocb', 'group_type', 'event', 'outgdist', 'custdist',
       'area', 'nikom', 'type', 'new_code', 'relay', 'component', 'road',
       'lateral', 'time_eq', 'cust_num', 'cust_min', 'major', 'control',
       'weather', 'bk', 'bk_min', 'sp', 'sp_min', 'nb', 'nb_min', 'CrossDist'],
      dtype='object')

In [143]:
def insertToDB(month, tableName):
    try:
        print()
        print(f'Insert data to "{tableName}" table')
        df[df['date'].dt.month == month].to_sql(tableName, con, if_exists='append', index=False)
        print('Success^^')
    except:
        print("May be data error then they can't be imported to MySQL !!!")
        raise

In [144]:
def checkNumberRecords(month, tableName):
    rawDataRecords = df[(df['date'].dt.year == int(yearSelected)) & (df['date'].dt.month == month)]['id'].count()
    print(f'Raw {yearSelected}/{month} data records : {rawDataRecords}')
    sql = f'select count(*) from {tableName} where year(date)={yearSelected} and month(date)={month}'
    dbDataRecords = con.execute(sql).fetchall()[0][0]
    print()
    print('Check number of records')
    print(f'{tableName} table {yearSelected}/{month} data records : {dbDataRecords}')
    if rawDataRecords == dbDataRecords:
        print(f'Successfully append to {tableName} ^^')
    else:
        print(f'Unsuccessfullt append to {tableName}, please resolve errors !!!')

### --- Pending here ---

In [72]:
tableNames = {1: 'outage_event_db', 2: 'outage_event_db_15days'}
selectTable = int(input(
'''Please type 1 or 2 for selecting table to dump
\t1 to outage_event_db
\t2 to outage_event_db_15days
Select table here : ''') or '0')
print()
if selectTable > 0 and selectTable < 3:
    selectMonth = int(input('Please select month between 1 to 12\nSelect table here : ') or '0')
    if selectMonth > 0 and selectMonth < 13:
        insertToDB(selectMonth, tableNames[selectTable])
        checkNumberRecords(selectMonth, tableNames[selectTable])
        selectTable = None
        selectMonth = None
    else:
        print('Please run this cell again then select month between 1 or 12 !!!')
else:
    print('Please run this cell again then select only 1 or 2 !!!')

Please type 1 or 2 for selecting table to dump
	1 to outage_event_db
	2 to outage_event_db_15days
Select table here : 1

Please select month between 1 to 12
Select table here : 9

Insert data to "outage_event_db" table
Success^^
Raw 2020/9 data records : 1372

Check number of records
outage_event_db table 2020/9 data records : 1372
Successfully append to outage_event_db ^^


## Addition

In [74]:
tableNames = {1: 'outage_event_db', 2: 'outage_event_db_15days'}
selectTable = int(input(
'''Please type 1 or 2 for selecting table to dump
\t1 to outage_event_db
\t2 to outage_event_db_15days
Select table here : ''') or '0')
print()
if selectTable > 0 and selectTable < 3:
    sql = f'select date from {tableNames[selectTable]} order by date desc limit 1'
    print(f'Lasted date of {tableNames[selectTable]} table (y-m-d) : {con.execute(sql).fetchall()[0][0]}')
else:
    print('Please run this cell again then select only 1 or 2 !!!')

Please type 1 or 2 for selecting table to dump
	1 to outage_event_db
	2 to outage_event_db_15days
Select table here : 2

Lasted date of outage_event_db_15days table (y-m-d) : 2020-09-30
